<a href="https://colab.research.google.com/github/monsterhunters/downloader/blob/main/ANB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Gdrive Mount
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# @title <img src="https://anubisscan.com/wp-content/uploads/2022/05/Anubis-Scan-3.png" width="200px">


import os, json
import requests
from urllib.parse import urljoin
from bs4 import BeautifulSoup
from ipywidgets import interact, widgets, HBox
from urllib.parse import urlparse
from IPython.display import clear_output
from IPython.display import display, HTML


# Function to extract image links from the script content
def extract_image_links(script_content):
    if script_content:
        # Find the JSON object within the script content
        start_index = script_content.find('{')
        end_index = script_content.rfind('}') + 1
        json_content = script_content[start_index:end_index]

        try:
            # Load the JSON object
            data = json.loads(json_content)

            # Extract the image links from the 'images' field
            if 'sources' in data:
                images = data['sources'][0].get('images', [])
                return images
        except json.JSONDecodeError:
            pass
    return []

download_path = "" # @param {type:"string"}
os.chdir(download_path)


# URL of the webpage
url = "" # @param {type:"string"}
parsed_url = urlparse(url)
base_url = parsed_url.scheme + "://" + parsed_url.netloc

root = url.split("/")[-2]
os.makedirs(root, exist_ok=True)
os.chdir(root)

response = requests.get(url)
html_content = response.text
soup = BeautifulSoup(html_content, 'html.parser')
chapter_list_items = soup.find_all('li', {'data-num': True})
#print(chapter_list_items)

chapter_numbers = []
for chapter_item in chapter_list_items:
    chapter_link = chapter_item.find('a')['href']
    chapter_number_text = chapter_link.split('/')[-2].split('-')[-1]  # Extracting the chapter number from the URL
    if chapter_number_text.isdigit():
        chapter_numbers.append(int(chapter_number_text))
    elif chapter_number_text.upper() == "END":
        chapter_numbers.append("END")






max_val = max(chapter_numbers)
min_val = min(chapter_numbers)

# Create a button to stop the download process
stop_button = widgets.Button(description="Stop Download")
stop_button.layout.width = '20%'

# Flag to indicate whether the download process should continue
continue_download = True

# Define the callback function for the stop button click event
def on_stop_button_click(b):
    global continue_download
    continue_download = False

stop_button.on_click(on_stop_button_click)

def extract_image_links(script_content):
    if script_content:
        start_index = script_content.find('{')
        end_index = script_content.rfind('}') + 1
        json_content = script_content[start_index:end_index]

        try:
            data = json.loads(json_content)
            if 'sources' in data:
                images = data['sources'][0].get('images', [])
                return images
        except json.JSONDecodeError:
            pass
    return []

def download_images(image_links, directory):
    os.makedirs(directory, exist_ok=True)
    os.chdir(directory)
    for link in image_links:
       !wget -q --show-progress $link
    clear_output()
    print('All files have been downloaded!')
       #filename = os.path.basename(link)
       #response = requests.get(link)
       #if response.status_code == 200:
       #    with open(filename, 'wb') as f:
       #        f.write(response.content)
       #        print(f"Downloaded {filename}")
       #else:
       #    print(f"Failed to download {filename}")
    os.chdir("../")

def download_chapters(min_chapter, max_chapter):
    global continue_download
    continue_download = True

    # Iterate over each list item and extract chapter details
    for chapter_item in chapter_list_items:
        # Check if the download process should continue
        if not continue_download:
            print("Download stopped.")
            return

        chapter_number_text = chapter_item.find('span', class_='chapternum').text.strip()
        # Check if the chapter number is in the expected format
        if not chapter_number_text.startswith('Chapter'):
            continue
        chapter_number_parts = chapter_number_text.split()
        if len(chapter_number_parts) < 2:
            continue
        chapter_number = chapter_number_parts[1]
        if not chapter_number.isdigit():
            continue
        chapter_number = int(chapter_number)
        chapter_date = chapter_item.find('span', class_='chapterdate').text.strip()
        chapter_link = chapter_item.find('a')['href']

        # Check if the chapter number is within the selected range
        if min_chapter <= chapter_number <= max_chapter:
            # Construct the complete URL for the chapter page
            chapter_url = urljoin(base_url, chapter_link)

            # Fetch the HTML content of the chapter page
            chapter_response = requests.get(chapter_url)
            if chapter_response.status_code == 200:
                chapter_html_content = chapter_response.text

                # Parse the HTML of the chapter page using BeautifulSoup
                chapter_soup = BeautifulSoup(chapter_html_content, 'html.parser')

                # Find the script tag containing the desired content
                script_tag = chapter_soup.find('script', string=lambda x: 'ts_reader.run' in str(x))
                script_content = script_tag.text if script_tag else None

                # Extract image links from the script content
                image_links = extract_image_links(script_content)
                #print(image_links)

                # Download images
                directory = chapter_url.split("/")[-2]
                download_images(image_links, directory)

                # Print chapter details
                print(f"Chapter {chapter_number}: {chapter_date} - {chapter_url}")
            else:
                print(f"Failed to fetch chapter page: {chapter_url}")

# Create a slider widget
slider = widgets.IntRangeSlider(min=min_val, max=max_val, step=1, value=(min_val, max_val))
slider.layout.width = '80%'

# Create a download button
button = widgets.Button(description="Set Chapter")
button.layout.width = '20%'

# Define the callback function for the button click event
def on_button_click(b):
    min_chapter, max_chapter = slider.value
    clear_output()
    # Open the file in write mode
    with open("/content/config.txt", "w") as file:
        # Write "Hello, World!" to the file
        file.write(str(download_path)+"\n"+
                   str(url)+"\n"+
                   str(min_chapter)+"\n"+
                   str(max_chapter))
    print("Chapter "+str(min_chapter)+" to "+str(max_chapter)+" has been set....")

    print('Run below cell to start download!!!')
    #download_chapters(min_chapter, max_chapter)


button.on_click(on_button_click)

# Display the slider, download button, and stop button
print('Set chapters to download')
display(slider)
display(button)
#display(HBox([button, stop_button]))


In [ ]:
# @title RUN

import os, json
import requests
from urllib.parse import urljoin
from bs4 import BeautifulSoup
from ipywidgets import interact, widgets, HBox
from urllib.parse import urlparse
from IPython.display import clear_output
from IPython.display import display, HTML


# Function to extract image links from the script content
def extract_image_links(script_content):
    if script_content:
        # Find the JSON object within the script content
        start_index = script_content.find('{')
        end_index = script_content.rfind('}') + 1
        json_content = script_content[start_index:end_index]

        try:
            # Load the JSON object
            data = json.loads(json_content)

            # Extract the image links from the 'images' field
            if 'sources' in data:
                images = data['sources'][0].get('images', [])
                return images
        except json.JSONDecodeError:
            pass
    return []



# Open the file in read mode
with open('/content/config.txt', 'r') as file:
    # Initialize a counter to keep track of lines read
    line_count = 0

    # Iterate through each line in the file
    for line in file:
        # Increment the line count
        line_count += 1

        # If it's the third line, print it and break the loop
        if line_count == 1:
          download_path = line.strip()
        elif line_count == 2:
          url = line.strip()
        elif line_count == 3:
          min_chapter = int(line.strip())
        else:
          max_chapter = int(line.strip())



#url = "https://anubisscan.com/manga/ill-eat-your-mom-first/"
#download_path = "/content/sample_data"
#min_chapter = 2
#max_chapter = 3
os.chdir(download_path)


# URL of the webpage

parsed_url = urlparse(url)
base_url = parsed_url.scheme + "://" + parsed_url.netloc

root = url.split("/")[-2]
os.makedirs(root, exist_ok=True)
os.chdir(root)

response = requests.get(url)
html_content = response.text
soup = BeautifulSoup(html_content, 'html.parser')
chapter_list_items = soup.find_all('li', {'data-num': True})
#print(chapter_list_items)

chapter_numbers = []
for chapter_item in chapter_list_items:
    chapter_link = chapter_item.find('a')['href']
    chapter_number_text = chapter_link.split('/')[-2].split('-')[-1]  # Extracting the chapter number from the URL
    if chapter_number_text.isdigit():
        chapter_numbers.append(int(chapter_number_text))
    elif chapter_number_text.upper() == "END":
        chapter_numbers.append("END")




# Flag to indicate whether the download process should continue
continue_download = True

def extract_image_links(script_content):
    if script_content:
        start_index = script_content.find('{')
        end_index = script_content.rfind('}') + 1
        json_content = script_content[start_index:end_index]

        try:
            data = json.loads(json_content)
            if 'sources' in data:
                images = data['sources'][0].get('images', [])
                return images
        except json.JSONDecodeError:
            pass
    return []

def download_images(image_links, directory):
    os.makedirs(directory, exist_ok=True)
    os.chdir(directory)
    for link in image_links:
       !wget -q --show-progress $link
    clear_output()
    print('All files have been downloaded!')
       #filename = os.path.basename(link)
       #response = requests.get(link)
       #if response.status_code == 200:
       #    with open(filename, 'wb') as f:
       #        f.write(response.content)
       #        print(f"Downloaded {filename}")
       #else:
       #    print(f"Failed to download {filename}")
    os.chdir("../")

def download_chapters(min_chapter, max_chapter):
    global continue_download
    continue_download = True

    # Iterate over each list item and extract chapter details
    for chapter_item in chapter_list_items:
        # Check if the download process should continue
        if not continue_download:
            print("Download stopped.")
            return

        chapter_number_text = chapter_item.find('span', class_='chapternum').text.strip()
        # Check if the chapter number is in the expected format
        if not chapter_number_text.startswith('Chapter'):
            continue
        chapter_number_parts = chapter_number_text.split()
        if len(chapter_number_parts) < 2:
            continue
        chapter_number = chapter_number_parts[1]
        if not chapter_number.isdigit():
            continue
        chapter_number = int(chapter_number)
        chapter_date = chapter_item.find('span', class_='chapterdate').text.strip()
        chapter_link = chapter_item.find('a')['href']

        # Check if the chapter number is within the selected range
        if min_chapter <= chapter_number <= max_chapter:
            # Construct the complete URL for the chapter page
            chapter_url = urljoin(base_url, chapter_link)

            # Fetch the HTML content of the chapter page
            chapter_response = requests.get(chapter_url)
            if chapter_response.status_code == 200:
                chapter_html_content = chapter_response.text

                # Parse the HTML of the chapter page using BeautifulSoup
                chapter_soup = BeautifulSoup(chapter_html_content, 'html.parser')

                # Find the script tag containing the desired content
                script_tag = chapter_soup.find('script', string=lambda x: 'ts_reader.run' in str(x))
                script_content = script_tag.text if script_tag else None

                # Extract image links from the script content
                image_links = extract_image_links(script_content)
                #print(image_links)

                # Download images
                directory = chapter_url.split("/")[-2]
                download_images(image_links, directory)

                # Print chapter details
                print(f"Chapter {chapter_number}: {chapter_date} - {chapter_url}")
            else:
                print(f"Failed to fetch chapter page: {chapter_url}")


# Define the callback function for the button click event
def main(min_chapter, max_chapter):
    download_chapters(min_chapter, max_chapter)

main(min_chapter, max_chapter)
